In [1]:
# imports
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import os
import pickle

import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm_notebook
from sklearn.preprocessing import StandardScaler
from sklearn.svm import NuSVR, SVR
from sklearn.metrics import mean_absolute_error
pd.options.display.precision = 15

import lightgbm as lgb
import xgboost as xgb
import time
import datetime
from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold, GroupKFold, GridSearchCV, train_test_split, TimeSeriesSplit
from sklearn import metrics
from sklearn import linear_model
import gc
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

import eli5
import shap
from IPython.display import HTML
import json
import altair as alt

import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline
import multiprocessing
import warnings
warnings.simplefilter('ignore')
alt.renderers.enable('notebook')
from functions import save_features2, LabelEncoderPopularity,load_data
from lib.training import *

In [2]:
from catenc.utils import *

## Load base dataset


In [3]:
data = pd.read_pickle("./pickles/data_all_features.pkl")


In [4]:
fraud = data['isFraud']
fraud = fraud.replace({'test':-1})
fraud = fraud.astype(np.int8)
data['isFraud'] = fraud

In [5]:
with pd.HDFStore('./hdf/Store.h5',complib='blosc:blosclz') as store:
    categorial_features = list(store['initial/categorial_features'].values.flatten())
    numerical_features = list(store['initial/numerical_features'].values.flatten())

## Apply Additional Features

In [6]:
with pd.HDFStore('./hdf/Store.h5',complib='blosc:blosclz') as store:
    num_features = store.get_node('encodedFeatures')
    for feature in num_features:
        feature_name = feature._v_name
        # here would be assession of feature introduction
        q = store.select(feature._v_pathname)
        if len(q[q.index.duplicated()]) != 0:
            print('Problem', feature_name)
            q.drop(q[q.index.duplicated()].index, axis=0, inplace=True)
        data[feature_name] = q
    numerical_features += ['BrowserAge', 'BrowserVersion', 'screen_height', 'screen_width']
    categorial_features +=  ['Browser',  'OS', 'OSVersion', 'device_name', 'device_version']
    
categorial_features.remove('id_30')
categorial_features.remove('id_31')
data.drop(['id_30','id_31'], axis=1, inplace=True)

## Add Engineering features

In [7]:
with pd.HDFStore('./hdf/engineering.h5', 'r') as store:
    numerical = store.get_node('numerical')
    for feature in numerical:
        feature_name = feature._v_name
        # here would be assession of feature introduction
        q = store.select(feature._v_pathname)
        if len(q[q.index.duplicated()]) != 0:
            print('Problem', feature_name)
            q.drop(q[q.index.duplicated()].index, axis=0, inplace=True)
        data[feature_name] = q
        numerical_features += [feature_name]
    categorial = store.get_node('categorial')
    for feature in categorial:
        feature_name = feature._v_name
        # here would be assession of feature introduction
        q = store.select(feature._v_pathname)
        if len(q[q.index.duplicated()]) != 0:
            print('Problem', feature_name)
            q.drop(q[q.index.duplicated()].index, axis=0, inplace=True)
        data[feature_name] = q
        categorial_features += [feature_name]        
    
    

In [8]:
data.isFraud.value_counts()

 0    569877
-1    506691
 1     20663
Name: isFraud, dtype: int64

In [9]:
data.head()

,isFraud,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,...,D11__device_name,P_emaildomain_bin__C2,addr1__card1,card1__card5,card2__dist1,card2__id_20,card5__P_emaildomain_bin,device_name__P_emaildomain_bin,id_02__D8,id_02__id_20
TransactionID,,,,,,,,,,,,,,,,,,,,,
2987000,0,68.5,W,13926,NaN,150.0,discover,142.0,credit,315.0,...,13.0_nan,nan_1.0,315.0_13926,13926_142.0,nan_19.0,nan_nan,142.0_nan,nan_nan,nan_nan,nan_nan
2987001,0,29.0,W,2755,404.0,150.0,mastercard,102.0,credit,325.0,...,nan_nan,google_1.0,325.0_2755,2755_102.0,404.0_nan,404.0_nan,102.0_google,nan_google,nan_nan,nan_nan
2987002,0,59.0,W,4663,490.0,150.0,visa,166.0,debit,330.0,...,315.0_nan,microsoft_1.0,330.0_4663,4663_166.0,490.0_287.0,490.0_nan,166.0_microsoft,nan_microsoft,nan_nan,nan_nan
2987003,0,50.0,W,18132,567.0,150.0,mastercard,117.0,debit,476.0,...,nan_nan,yahoo_5.0,476.0_18132,18132_117.0,567.0_nan,567.0_nan,117.0_yahoo,nan_yahoo,nan_nan,nan_nan
2987004,0,50.0,H,4497,514.0,150.0,mastercard,102.0,credit,420.0,...,nan_2.0,google_1.0,420.0_4497,4497_102.0,514.0_nan,514.0_144.0,102.0_google,2.0_google,70787.0_nan,70787.0_144.0


In [33]:
# tdata = data[data['isFraud'] >= 0].drop(['Date'],axis=1)
X = data[data['isFraud'] >= 0].drop(['isFraud','Date'],axis=1)
y = data[data['isFraud'] >= 0]['isFraud']#.astype(np.bool)
X_test = data[data['isFraud'] < 0].drop(['isFraud','Date'],axis=1)   

In [11]:
list(data.columns)

['isFraud',
 'TransactionAmt',
 'ProductCD',
 'card1',
 'card2',
 'card3',
 'card4',
 'card5',
 'card6',
 'addr1',
 'addr2',
 'dist1',
 'P_emaildomain',
 'R_emaildomain',
 'C1',
 'C2',
 'C4',
 'C5',
 'C6',
 'C7',
 'C8',
 'C9',
 'C10',
 'C11',
 'C12',
 'C13',
 'C14',
 'D1',
 'D2',
 'D3',
 'D4',
 'D5',
 'D6',
 'D8',
 'D9',
 'D10',
 'D11',
 'D12',
 'D13',
 'D14',
 'D15',
 'M2',
 'M3',
 'M4',
 'M5',
 'M6',
 'M7',
 'M8',
 'M9',
 'V3',
 'V4',
 'V5',
 'V6',
 'V7',
 'V8',
 'V9',
 'V10',
 'V11',
 'V12',
 'V13',
 'V17',
 'V19',
 'V20',
 'V29',
 'V30',
 'V33',
 'V34',
 'V35',
 'V36',
 'V37',
 'V38',
 'V40',
 'V44',
 'V45',
 'V46',
 'V47',
 'V48',
 'V49',
 'V51',
 'V52',
 'V53',
 'V54',
 'V56',
 'V58',
 'V59',
 'V60',
 'V61',
 'V62',
 'V63',
 'V64',
 'V69',
 'V70',
 'V71',
 'V72',
 'V73',
 'V74',
 'V75',
 'V76',
 'V78',
 'V80',
 'V81',
 'V82',
 'V83',
 'V84',
 'V85',
 'V87',
 'V90',
 'V91',
 'V92',
 'V93',
 'V94',
 'V95',
 'V96',
 'V97',
 'V99',
 'V100',
 'V126',
 'V127',
 'V128',
 'V130',
 'V131',
 'V138',
 'V139',
 'V140',
 'V143',
 'V145',
 'V146',
 'V147',
 'V149',
 'V150',
 'V151',
 'V152',
 'V154',
 'V156',
 'V158',
 'V159',
 'V160',
 'V161',
 'V162',
 'V163',
 'V164',
 'V165',
 'V166',
 'V167',
 'V169',
 'V170',
 'V171',
 'V172',
 'V173',
 'V175',
 'V176',
 'V177',
 'V178',
 'V180',
 'V182',
 'V184',
 'V187',
 'V188',
 'V189',
 'V195',
 'V197',
 'V200',
 'V201',
 'V202',
 'V203',
 'V204',
 'V205',
 'V206',
 'V207',
 'V208',
 'V209',
 'V210',
 'V212',
 'V213',
 'V214',
 'V215',
 'V216',
 'V217',
 'V219',
 'V220',
 'V221',
 'V222',
 'V223',
 'V224',
 'V225',
 'V226',
 'V227',
 'V228',
 'V229',
 'V231',
 'V233',
 'V234',
 'V238',
 'V239',
 'V242',
 'V243',
 'V244',
 'V245',
 'V246',
 'V247',
 'V249',
 'V251',
 'V253',
 'V256',
 'V257',
 'V258',
 'V259',
 'V261',
 'V262',
 'V263',
 'V264',
 'V265',
 'V266',
 'V267',
 'V268',
 'V270',
 'V271',
 'V272',
 'V273',
 'V274',
 'V275',
 'V276',
 'V277',
 'V278',
 'V279',
 'V280',
 'V282',
 'V283',
 'V285',
 'V287',
 'V288',
 'V289',
 'V291',
 'V292',
 'V294',
 'V303',
 'V304',
 'V306',
 'V307',
 'V308',
 'V310',
 'V312',
 'V313',
 'V314',
 'V315',
 'V317',
 'V322',
 'V323',
 'V324',
 'V326',
 'V329',
 'V331',
 'V332',
 'V333',
 'V335',
 'V336',
 'V338',
 'NanTransactionCount',
 'id_01',
 'id_02',
 'id_03',
 'id_04',
 'id_05',
 'id_06',
 'id_07',
 'id_08',
 'id_09',
 'id_10',
 'id_11',
 'id_12',
 'id_13',
 'id_14',
 'id_15',
 'id_16',
 'id_17',
 'id_18',
 'id_19',
 'id_20',
 'id_21',
 'id_22',
 'id_23',
 'id_24',
 'id_25',
 'id_26',
 'id_27',
 'id_28',
 'id_29',
 'id_32',
 'id_33',
 'id_34',
 'id_35',
 'id_36',
 'id_37',
 'id_38',
 'DeviceType',
 'DeviceInfo',
 'NanIdentityCount',
 'Date',
 '_Weekdays',
 '_Hours',
 '_Days',
 'P_emaildomain_bin',
 'P_emaildomain_suffix',
 'R_emaildomain_bin',
 'R_emaildomain_suffix',
 'Browser',
 'BrowserAge',
 'BrowserVersion',
 'OS',
 'OSVersion',
 'device_name',
 'device_version',
 'screen_height',
 'screen_width',
 'D11__device_name',
 'P_emaildomain_bin__C2',
 'addr1__card1',
 'card1__card5',
 'card2__dist1',
 'card2__id_20',
 'card5__P_emaildomain_bin',
 'device_name__P_emaildomain_bin',
 'id_02__D8',
 'id_02__id_20']

In [17]:
cols = [f for f in categorial_features if f in data.columns.values]
encoder_names = ("JamesSteinEncoder", "CatBoostEncoder")
Encoder = DoubleValidationEncoderNumerical(cols, encoder_names)

In [31]:
# X_train, X_test, y_train, y_test = train_test_split(data.drop("isFraud", axis=1), data["isFraud"],
#                                                     test_size=0.4, shuffle=False)
# X_train, X_test = X_train.reset_index(drop=False), X_test.reset_index(drop=False)
# y_train, y_test = np.array(y_train), np.array(y_test)

In [32]:
y_train

array([ 0,  0,  0, ..., -1, -1, -1], dtype=int8)

In [36]:
y.values

array([0, 0, 0, ..., 0, 0, 0], dtype=int8)

In [ ]:
X = X.reset_index(drop=False)
# y = np.array(y.reset_index(drop=False))
# X_test = X_test.reset_index(drop=False)
X_emb = Encoder.fit_transform(X, y.values)


In [ ]:
X_emb

In [22]:
y.shape

(590540,)

In [ ]:
X_test = Encoder.transform(X_test)

In [ ]:
X_test.head()

In [ ]:
Encoder.storage